In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data = "Price/AAPL.csv"

df = pd.read_csv(data)
print(df.shape)
df.tail(21)

(2218, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
2197,2018-09-24,216.820007,221.259995,216.630005,220.789993,220.789993,27693400
2198,2018-09-25,219.750000,222.820007,219.699997,222.190002,222.190002,24554400
2199,2018-09-26,221.000000,223.750000,219.759995,220.419998,220.419998,23984700
2200,2018-09-27,223.820007,226.440002,223.539993,224.949997,224.949997,30181200
2201,2018-09-28,224.789993,225.839996,224.020004,225.740005,225.740005,22929400
2202,2018-10-01,227.949997,229.419998,226.350006,227.259995,227.259995,23600800
2203,2018-10-02,227.250000,230.000000,226.630005,229.279999,229.279999,24788200
2204,2018-10-03,230.050003,233.470001,229.779999,232.070007,232.070007,28654800
2205,2018-10-04,230.779999,232.350006,226.729996,227.990005,227.990005,32042000
2206,2018-10-05,227.960007,228.410004,220.580002,224.289993,224.289993,33580500


In [4]:
def create_data(df, training_length):
    adj_close = df['Adj Close'].values
    time_steps = adj_close.shape[0]
    features = 1
    sequences = time_steps - training_length + 1
    x = np.zeros((sequences-1, training_length, features))
    y = np.zeros((sequences-1))
    for seq in range(sequences-1):
        x[seq, :, :] = adj_close[seq:seq+training_length].reshape(training_length, features)   
        y[seq] = adj_close[seq+training_length]
    return x, y

In [5]:
x, y = create_data(df, 40)
y.dtype

dtype('float64')

In [10]:
model = Sequential()
model.add(LSTM(64,
               activation='sigmoid',
               recurrent_activation='sigmoid',
               return_sequences=True, 
               input_shape=(40,1)))
model.add(Dropout(0.5))
model.add(LSTM(64,
              return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,
              return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['mae', 'mse'])


In [11]:
model.fit(x, y, epochs=40, batch_size=32)

Epoch 1/40
2178/2178 [==============================] - 40s 18ms/step - loss: 6368.1668 - mean_absolute_error: 62.8881 - mean_squared_error: 6368.1668
Epoch 2/40
2178/2178 [==============================] - 30s 14ms/step - loss: 2803.1298 - mean_absolute_error: 42.9923 - mean_squared_error: 2803.1298
Epoch 3/40
2178/2178 [==============================] - 29s 13ms/step - loss: 2811.7523 - mean_absolute_error: 43.6739 - mean_squared_error: 2811.7523
Epoch 4/40
2178/2178 [==============================] - 29s 13ms/step - loss: 2779.9187 - mean_absolute_error: 43.2253 - mean_squared_error: 2779.9187
Epoch 5/40
2178/2178 [==============================] - 28s 13ms/step - loss: 2822.7787 - mean_absolute_error: 43.6991 - mean_squared_error: 2822.7787
Epoch 6/40
2178/2178 [==============================] - 29s 13ms/step - loss: 2819.3192 - mean_absolute_error: 43.4071 - mean_squared_error: 2819.3192
Epoch 7/40
2178/2178 [==============================] - 29s 14ms/step - loss: 2804.6665 - mean

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 40, 64)            16896     
_________________________________________________________________
dropout_15 (Dropout)         (None, 40, 64)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 40, 64)            33024     
_________________________________________________________________
dropout_16 (Dropout)         (None, 40, 64)            0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
__________